# USV/UAV Trajectory Analysis

This is the supplementary materials for CuRL-Transfer project.

CSV formate:
* Follow movingpandas examples (X, Y, t, trajectory_id, ...)
* Framerate: 10 fps
* TODO crs
* TODO overlay a background (geo image or other background?)


In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString
import numpy as np

import sys
sys.path.append("..")
import movingpandas as mpd
# mpd.show_versions()

from datetime import datetime

import warnings
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt

## Reading CSVs

read csv into
1. wamvs: dictionary of GeoDataFrame
2. trajs: dictionary of Trajectory

In [ ]:
boats = ['wamv1', 'wamv2', 'wamv3', 'wamv4', 'wamv5']
wamvs = {}
trajs = {}

for boat in boats:
    df = pd.read_csv('data/bory_boat/' + boat + '_gps.csv', delimiter=',')
    df['t'] = pd.to_datetime(df['t'], unit='ns')
    wamv = df[['X', 'Y', 'trajectory_id', 't']]
    wamv['geometry'] = df.apply(lambda row: Point(row['X'], row['Y']), axis=1)

    gdf = gpd.GeoDataFrame(wamv, geometry='geometry', crs='EPSG:4326')
    gdf = gdf.to_crs(epsg=32649) # Convert to UTM zone 49N for Taiwan

    trajs[boat] = mpd.Trajectory(gdf, 'trajectory_id', t='t')
    wamvs[boat] = gdf

In [ ]:
wamvs['wamv1']

## GeoPandas

In [ ]:
merged_gdf = pd.concat([wamvs['wamv1'], wamvs['wamv2'], wamvs['wamv3'], wamvs['wamv4'], wamvs['wamv5']], axis=0)
merged_gdf

## MovingPandas

documentation : https://movingpandas.readthedocs.io/en/main/trajectorycollection.html

### speed 

In [ ]:
traj_collection = mpd.TrajectoryCollection(merged_gdf, 'trajectory_id', t='t')
traj_collection.add_speed()

In [ ]:
traj_collection.get_max('speed')

In [ ]:
traj_collection.plot(column='speed', legend=True, figsize=(9,5))

## direction

In [ ]:
traj_collection.add_direction()
traj_collection.plot(column='direction', legend=True, figsize=(9,5))

## location

In [ ]:
traj_collection.get_start_locations()

In [ ]:

traj_collection.get_end_locations()

## segments between times

In [ ]:
t1 = datetime(1970,1,1,0,14,14)
t2 = datetime(1970,1,1,0,15,44)
traj_collection.get_segments_between(t1, t2).plot(column='speed', legend=True, figsize=(9,5))

# Demos

## Demo1: total distance and avg speed

compute the total distance and average speed of each trajectory.

In [ ]:
summary = []
for traj in traj_collection.trajectories:
    #############  code below  #############
    # Hint: avg_speed = length / duration

    #############  code above  #############
    summary.append({
        "trajectory_id": traj.id,
        "total_distance(m)": length,
        "avg_speed(m/s)": avg_speed,
    })

summary_df = pd.DataFrame(summary)
print(summary_df)

## Demo2: Mark all the stop points on plot

mark all point whose speed < 0.1 m/s with red dot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
for traj in traj_collection.trajectories:
    traj.plot(ax=ax, linewidth=2, alpha=0.7, label=traj.id)
    #############  code below  #############
    # Hint: Chat-GPT -> get points with speed < 0.1 m/s, save to GeoSeries

    #############  code above  #############

ax.legend()
plt.title('Trajectory with low-speed points (speed < 0.1 m/s)')
plt.show()

## Demo3: Mark all the turning point on plot

Mark all the turning point (direction change > 45 degrees & move < 2 meters within 5 tracked point)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
turning_points = {}
TURN_ANGLE_THRESHOLD = 45
MOVE_DISTANCE_THRESHOLD = 2
SHIFT = 5

for traj in traj_collection.trajectories:
    #############  code below  #############
    # Hint: calculate shifted geometry and direction, then find turning points and store them in a GeoSeries

    #############  code above  #############

ax.legend()
plt.title('Trajectories with turning points (direction change > 45 degrees)')
plt.xlabel("Easting (m)")
plt.ylabel("Northing (m)")
plt.show()
